In [215]:
cd home/Gourav/Models

[Errno 2] No such file or directory: 'home/Gourav/Models'
/home/Gourav/Models


In [216]:
print('Starting of MLP training')
import logging
import datetime
import time
import os

import time
start_time = time.time()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# In[10]:

print(os.getcwd())
if not os.path.exists('/home/Gourav/Models/approach_logs/MLP_classifier/'):
    os.makedirs('/home/Gourav/Models/approach_logs/MLP_classifier/')

Starting of MLP training
/home/Gourav/Models


In [ ]:
import time
start_time = time.time()

In [ ]:
import datetime
import numpy as np
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql import DataFrameStatFunctions as stat
import numpy as np
import pandas as pd
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql.types import *
from pyspark.sql import Window

import requests
from collections import defaultdict
from pymongo import MongoClient
from tqdm import tqdm
from pyspark.sql import DataFrameStatFunctions as stat
from pyspark.sql import Window
import pyspark.sql.functions as F
import os
import time
import json
import pickle

NIS_DATA_BASE_PATH = "gs://nis-segment-datasource-v3/processed/"
NIS_OLD_DATA_BASE_PATH = "gs://nis-localytics-datasource/processed/"

conf = SparkConf().setAll([('spark.driver.memory', '100g'), ('spark.broadcast.blockSize', '50m'), ("spark.executor.instances", '30')])
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)


In [ ]:
def get_path(dates, prefix, padding=None):
    dates_ = dates + []
    if padding:
        st_date, ed_date = sorted(dates)[0], sorted(dates)[-1]
        for i in range(1, 4):
            d = (datetime.datetime.strptime(ed_date, date_fmt) + datetime.timedelta(days=i)).strftime(date_fmt)
            if d < datetime.datetime.today().strftime(date_fmt):
                dates_.append(d)
    dates_ = list(set(dates_) - set(["2019/02/17", "2019/02/18", "2019/05/28", "2019/06/03", "2019/07/02", "2019/07/03", "2019/07/04", "2019/11/13", "2019/11/14", "2020/02/22", "2020/03/31", "2020/04/16", "2020/04/18", "2020/05/11", "2021/05/13"]))
    paths = []
    for date in dates_:
        base_path = NIS_DATA_BASE_PATH
        if date < "2018/06/26":
            base_path = NIS_OLD_DATA_BASE_PATH
        paths.append(base_path + date + "/" + prefix + "/*.parquet")
    return paths

date_fmt = "%Y/%m/%d"
month_fmt = "%Y/%m"

def millis2date(x):
    try:
        if x < 15000000000:
            return datetime.datetime.fromtimestamp(x).strftime(date_fmt)
        else:
            return datetime.datetime.fromtimestamp(x / 1000.).strftime(date_fmt)
    except:
        return "1970/01/01"

def millis2month(x):
    try:
        if x < 15000000000:
            return datetime.datetime.fromtimestamp(x).strftime(month_fmt)
        else:
            return datetime.datetime.fromtimestamp(x / 1000.).strftime(month_fmt)
    except:
        return "1970/01"

millis2date_udf = F.udf(millis2date, StringType())
millis2month_udf = F.udf(millis2month, StringType())

def divide_maps(d1, d2):
    keys = set(d1.keys()).intersection(set(d2.keys()))
    res = {}
    for k in keys:
        res[k] = d1[k] * 1. / (d2[k] + 1e-10)
    return res

def timediff(y, x, date_fmt="%Y/%m/%d"): 
    end = datetime.datetime.strptime(y, date_fmt)
    start = datetime.datetime.strptime(x, date_fmt)
    delta = (end - start).days
    return delta

def monthdiff(y, x, month_fmt="%Y/%m"): 
    millis = y - x
    delta = millis / (1000 * 3600 * 24 * 30)
    return delta

timediff_udf = udf(timediff, IntegerType())
monthdiff_udf = udf(monthdiff, IntegerType())

def filter_platform(data, platform=None):
    if platform == "ANDROID":
        data = data.filter(data.platform == "ANDROID")
    elif platform == "IOS":
        data = data.filter(data.platform != "ANDROID")
    return data

def filter_category(data, categories=None):
    if categories:
        data = data.filter(data.categoryWhenEventHappened.isin(categories))
    return data

def filter_tenant(data, tenant=None):
    if tenant in ['hi', 'HINDI']:
        data = data.filter(data.tenant.isin(['hi', 'HINDI', 'Hindi', 'hindi']))
    elif tenant in ['en', 'ENGLISH']:
        data = data.filter(~data.tenant.isin(['hi', 'HINDI', 'Hindi', 'hindi']))
    return data


def filter_app(data, app_name=None):
    if 'appName' in data.columns:
        if app_name:
            data = data.filter(data.appName == app_name)
        else:
            data = data.filter((data.appName != "mini") & (data.appName != "crux"))
    return data


# In[7]:

In [ ]:
def get_raw_path(date, hours=None):
    paths = []
    base_path = NIS_RAW_DATA_BASE_PATH + date
    if not hours:
        return base_path + "/*/*.gz"
    for hour in hours:
        paths.append(base_path + "/" + str(hour).zfill(2) + "/*.gz")
    return ",".join(paths)

def process_raw_data(paths):
    def view_data_filters(x):
        x = x['properties']
        deviceid_filter = ('deviceId' in x) and (x['deviceId'] != '')
        time_filter = ('timeSpent' in x) and (int(x['timeSpent']) <= 100) and (int(x['timeSpent']) >= 0)
        return deviceid_filter and time_filter

    try:
        rdd = sc.textFile(paths) \
            .map(json.loads) \
            .filter(lambda x: "batch" in x).flatMap(lambda x: x["batch"]) \
            .filter(lambda x: ("event" in x) and (x["event"].lower() == "timespent-front")) \
            .filter(view_data_filters) \
            .map(lambda x: x['properties'])
        view_data = rdd.map(lambda x: (x['deviceId'], x['hashId'][:-2], x['timeSpent'])) \
            .toDF(['deviceId', 'hashId', 'timeSpent'])
        
        view_data = view_data.filter(view_data.timeSpent.isNotNull())
        #view_data = view_data.filter((getHashBucketUDF(view_data.deviceId) >= 16) & (getHashBucketUDF(view_data.deviceId) <= 25))
        view_data = view_data.groupby(view_data.deviceId, view_data.hashId) \
                             .agg(F.max(view_data.timeSpent).alias('overallTimeSpent'))
        
        return view_data
    except Exception as e:
        logger.warning("Error processing data: " + str(e))


In [ ]:
from Utils import *
def getNewsData(d1, d2):
    newsMap = getNewsInDates(d1, d2)
    hashIdList = list(newsMap.keys())

    hashIdsWithFilter = []
    for h in hashIdList:
        if 'newsLanguage' in newsMap[h] and newsMap[h]['newsLanguage'] == 'english' and newsMap[h]['publishGroupList'][0]['countryCode'] == 'IN':
            hashIdsWithFilter.append(h.split('-')[0])
    
    return hashIdsWithFilter, newsMap


In [ ]:
import logging

logger = logging.getLogger(str(datetime.datetime.today().date()))
hdlr = logging.FileHandler(
    '/home/Gourav/Models/approach_logs/MLP_classifier/' + str(datetime.datetime.today().date()) + '.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.DEBUG)

In [ ]:
def Log(s, flag=True):
    if flag:
        logger.info(s)
        print(s)

Log('----', flag=True)


In [ ]:
sampled_dates = [datetime.datetime(2023,2,1,0,0,0), datetime.datetime(2023,2,6,0,0,0),datetime.datetime(2023,2,9,0,0,0),datetime.datetime(2023,2,14,0,0,0),
                datetime.datetime(2023,2,18,0,0,0),datetime.datetime(2023,2,20,0,0,0), datetime.datetime(2023,2,25,0,0,0)]
sampled_dates

In [ ]:
train_days = 10
test_days = 7
embed_days = 30
n_days = train_days + test_days + embed_days
k = 0
st_date = (sampled_dates[k]+ datetime.timedelta(days=test_days)) - datetime.timedelta(days=n_days)


date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]
dates_str_embed = dates_str[0:embed_days]
dates_str_train = dates_str[embed_days:embed_days+train_days+1]
dates_str_test = dates_str[-test_days:]
print("embed data dates : ", dates_str_embed, len(dates_str_embed))
print("train data dates : ",dates_str_train, len(dates_str_train) )
print("test data dates : ",dates_str_test, len(dates_str_test))
# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000
print(dates[0], dates[-1])
hashIdsWithFilter, newsMap = getNewsData(dates[0], dates[-1])

In [ ]:
#need this for live data else skip if backtesting
NIS_RAW_DATA_BASE_PATH = "gs://inshorts-segment-raw/data/segment-raw-v5a/"

path = get_raw_path(datetime.datetime.now().strftime("%Y/%m/%d"))
today_data = process_raw_data(path)
today_data = today_data.filter(today_data.hashId.isin(hashIdsWithFilter))

In [ ]:
#generate datasets
def generate_data(datestr):
#     datestr = [d.strftime(date_fmt) for d in dates]
    paths = get_path(datestr, 'otherEvents')
#     print(sorted(paths), len(paths))
    data = sqlContext.read.parquet(*paths)

    data = filter_app(data, app_name=None)
    data = filter_tenant(data, tenant='en')

    # data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))

    data = data.select(data.deviceId, (F.split(data.hashId, '-')[0]).alias('hashId'), data.overallTimeSpent, data.notificationOpened, data.notificationShown, data.notificationType,data.eventName)
    data = data.filter( data.eventName.isin(['Notification Shown','Notification Opened']) )

    data = data.filter(data.hashId.isin(hashIdsWithFilter))
    
    data = data.select('deviceId', 'hashId' ,'overallTimeSpent', 'notificationOpened', 'notificationShown')\
            .groupby('deviceId', 'hashId')\
            .agg(F.max('overallTimeSpent').alias('timeSpent'), 
                 F.sum('notificationShown').alias('notiShown'),F.sum('notificationOpened').alias('notiOpened'))
    data = data.withColumn("notiShown",F.when(data.notiShown < data.notiOpened, 
                                                     data.notiOpened).otherwise(data.notiShown))
    return data

data_embed = generate_data(dates_str_embed)
data_train = generate_data(dates_str_train)
data_test = generate_data(dates_str_test)

In [ ]:
data_test

In [ ]:
#to be used when live data is needed
data = data.union(today_data)
data.cache()

In [ ]:
def popularity(data, newsMeanMap):
    Log("Computing news mean")
    newsMeandf = data.groupby(data.hashId).agg({'overallTimeSpent' : 'sum', 'hashId' : 'count'}).toPandas()

    for v in newsMeandf.values:
        newsMeanMap[v[0]] = newsMeanMap[v[0]][0] + v[1],  newsMeanMap[v[0]][1] + v[2]

newsMeanMap_train = defaultdict(lambda: (7, 1))
popularity(data, newsMe1banMap)

In [ ]:
data_embed.printSchema()

In [350]:
data_comb = data_embed.union(data_train)
#data_comb.groupBy('eventName').count().orderBy('count').show()

In [224]:
data_comb

DataFrame[deviceId: string, hashId: string, timeSpent: double, notiShown: bigint, notiOpened: bigint]

In [ ]:
data_comb.filter(data_comb.notificationOpened > 0).groupBy('eventName').count().orderBy('count').show()

In [ ]:
# whats happening in next few rowS?

In [ ]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA
import time

training_start = time.time()


# data_filtered = data.filter(F.udf(lambda hashId, noti: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))
# abcset=sqlContext.read.parquet("gs://pvtrough_asia_south1/tf_idf/devices_fulfilling_criteria")
# df_temp=abcset.filter(abcset.set_typ == 'roberta').select(['deviceid']).distinct()
# df_temp=df_temp.withColumnRenamed("deviceid","deviceId")

# data_filtered=data_filtered.join(df_temp,["deviceId"],"inner")

data_embed = data_embed.filter(data_embed.notiOpened>0)    


# Log("Training took %s minutes"%(int((time.time() - training_start) / 60)))

In [ ]:
d1 = data.select('deviceId').distinct().collect()
d2 = data_filtered.select('deviceId').distinct().collect()
d3 = df_temp.select('deviceId').distinct().collect()

In [ ]:
len(d2)

In [ ]:
from tqdm import tqdm
df_left = pd.DataFrame(columns = ['deviceId', 'cluster_left_out_date'])
left_out = set(d3) - set(d2)
pbar = tqdm(total=len(left_out))
cluster_left_out = (datetime.datetime.now() + datetime.timedelta(days=0)) .strftime("%Y_%m_%d") 
i = 0
for d in left_out:
    df_left.loc[i,'deviceId'] = d['deviceId']
    df_left.loc[i,'cluster_left_out_date'] = cluster_left_out
    i= i+1
    pbar.update(1)
pbar.close()

In [ ]:
dfl = df_left.copy()
print(dfl.head())

In [ ]:

path_leftout = 'gs://pvtrough_asia_south1/clustering/leftout_devices_roberta'
df_left = sqlContext.createDataFrame(dfl[['deviceId', 'cluster_left_out_date']])
df_left.coalesce(1).write.partitionBy('cluster_left_out_date').csv(path_leftout , sep=',', header=True,mode = 'append')

    

In [ ]:
#till here

In [ ]:
import torch.nn as nn
from torch.nn.parallel import DataParallel
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer , RobertaForSequenceClassification
from transformers import EarlyStoppingCallback


model_name = "roberta-base"
num_classes = 17
# model_path = "output/checkpoint-12000"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes,output_hidden_states=True)

# Move the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Check the number of available GPUs
if torch.cuda.device_count() > 1:
    # Specify which GPUs to use
    device_ids = [0, 1]  # Adjust the GPU IDs based on your system configuration
    model = DataParallel(model, device_ids=device_ids)
    print(isinstance(model, DataParallel))


In [ ]:
def get_embedding(model, tokenizer, text):
    model.eval()

    with torch.no_grad():
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt').to(device)
        outputs = model(input_ids=input_ids)
        hidden_states = outputs.hidden_states
        last_layer_hidden_states = hidden_states[-1]
        embeddings = torch.mean(last_layer_hidden_states, dim=1).squeeze().cpu().numpy()

    return embeddings

In [260]:
import os
import sys
import torch
import numpy as np

from transformers import RobertaModel, RobertaTokenizer

from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity


def Dim_reduction(sentences, tokenizer, model):
    '''
        This method will accept array of sentences, roberta tokenizer & model
        next it will call methods for dimention reduction
    '''

    vecs = []
    with torch.no_grad():

        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True,  max_length=64)
            inputs['input_ids'] = inputs['input_ids'].to(DEVICE)
            inputs['attention_mask'] = inputs['attention_mask'].to(DEVICE)

            hidden_states = model(**inputs, return_dict=True, output_hidden_states=True).hidden_states

            #Averaging the first & last hidden states
            output_hidden_state = (hidden_states[-1] + hidden_states[1]).mean(dim=1)

            vec = output_hidden_state.cpu().numpy()[0]

            vecs.append(vec)
    
#     print("35:",vecs)
    #Finding Kernal
    kernel, bias = compute_kernel_bias([vecs])
    kernel = kernel[:, :200]
    #If you want to reduce it to 128 dim
    #kernel = kernel[:, :128]
    embeddings = []
    embeddings = np.vstack(vecs)
#     print("43:", kernel, bias)
#     print("44:",embeddings)
    #Sentence embeddings can be converted into an identity matrix
    #by utilizing the transformation matrix
    embeddings = transform_and_normalize(embeddings, 
                kernel=kernel,
                bias=bias
            )
    return embeddings

In [252]:
import numpy as np

def transform_and_normalize(vecs, kernel, bias):
    """
        Applying transformation then standardize
    """
    if not (kernel is None or bias is None):
        vecs = (vecs + bias).dot(kernel)
    return normalize(vecs)
    
def normalize(vecs):
    """
        Standardization
    """
    return vecs / (vecs**2).sum(axis=1, keepdims=True)**0.5
    
def compute_kernel_bias(vecs):
    """
    Calculate Kernal & Bias for the final transformation - y = (x + bias).dot(kernel)
    """
    vecs = np.concatenate(vecs, axis=0)
    mu = vecs.mean(axis=0, keepdims=True)
    cov = np.cov(vecs.T)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(s**0.5))
    W = np.linalg.inv(W.T)
    return W, -mu

In [253]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = RobertaTokenizer.from_pretrained('C:/Models/roberta-base-nli-stsb-mean-tokens')
# model = RobertaModel.from_pretrained('C:/Models/roberta-base-nli-stsb-mean-tokens')

sentences = ['someone is slicing a onion', 'someone is carrying a fish']

embeddings = Dim_reduction(sentences, tokenizer, model)
# print(distance.cosine(embeddings[0],embeddings[1]))
print(f'Similarity score of two sentences {cosine_similarity([embeddings[0],embeddings[1]])[0][1]}')
print(f"New Dimension is {np.shape(embeddings)}")

Similarity score of two sentences 0.7305076834448454
New Dimension is (2, 100)


In [ ]:
embeddings[0]

# Checking for News embeddings just using title

In [ ]:
# reducing the size of embeddings using dimensionally reduction

In [ ]:
k=0
from pyspark.sql.functions import array, col
path_embedding = "gs://pvtrough_asia_south1/clustering/News_tilte_embedding"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
df_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)
# df_embedding.take(1)

In [ ]:
#generate news embeddings based on notification text
embedding_start = time.time()
document_embeddings_title={}

newsMapProcessed = {}
document_embeddings = {}
#try:
#    already_embedded = df_embedding.select('hid').distinct().collect()
#    already_embedded = [x['hid'] for x in already_embedded]
#except:
#    print("News Embeddings not found, generating new one")
already_embedded = []
for hId in tqdm(newsMap):
    
    h = hId.split('-')[0]
#     print(h)
    if h in already_embedded or (h not in hashIdsWithFilter) :
        continue
    else:    
        newsMapProcessed[h] = {}
        newsMapProcessed[h]['title'] = newsMap[hId]['title']
        newsMapProcessed[h]['content'] = newsMap[hId]['content']
        newsMapProcessed[h]['features'] = newsMapProcessed[h]['title'] #+ "." + newsMapProcessed[h]['content']
        document_embeddings[h] = get_embedding(model, tokenizer, newsMapProcessed[h]['features'])
        

        document_embeddings_title[h] = document_embeddings[h]
        

Log("Embedding took %s minutes"%(int((time.time() - embedding_start) / 60)))
    

100%|██████████| 24176/24176 [08:26<00:00, 47.70it/s]>         (153 + 35) / 188]

Embedding took 8 minutes


In [ ]:
len(document_embeddings_title)

15280

In [ ]:
path_embedding = "gs://pvtrough_asia_south1/clustering/News_tilte_embedding"+sampled_dates[k].strftime("%Y_%m_%d")+"/"


In [ ]:
if len(document_embeddings_title) > 0:
    df_em_pandas = pd.DataFrame(document_embeddings_title).T
    df_em_pandas['hid'] = df_em_pandas.index
    df_em_pandas['embed_date'] = sampled_dates[0]
    df_em_spark = (sqlContext.createDataFrame(df_em_pandas))
    df_em_spark.write.partitionBy('embed_date').csv(path_embedding , sep=',', header=True,mode = 'overwrite')

23/06/23 19:41:36 WARN org.apache.spark.scheduler.TaskSetManager: Stage 3459 contains a task of very large size (2158 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:
from pyspark.sql.functions import array, col


df_news_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)
df_news_embedding = df_news_embedding.distinct()
feat_cols = [str(x) for x in range(0,len(df_news_embedding.columns) - 2)]
df_embedding_exploded = df_news_embedding.distinct()
df_news_embedding = df_news_embedding.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))
df_news_embedding.take(1)


[Row(hid='gkefkzeg', embedding=[0.018744973465800285, 0.07465686649084091, 0.00890152808278799, -0.11559038609266281, -0.26225796341896057, 0.0443819984793663, 0.015611527487635612, 0.030492758378386497, 0.19157782196998596, -0.025312107056379318, -0.11552142351865768, -0.016290172934532166, 0.01366395503282547, -0.1699371635913849, 0.0643385648727417, -0.17743484675884247, 0.06688039004802704, -0.013621450401842594, 0.05057955160737038, 0.36965441703796387, -0.028559869155287743, 0.06138797476887703, -0.15975259244441986, -0.029601287096738815, 0.040057308971881866, 0.035204965621232986, 0.001116596395149827, 0.030557503923773766, 0.08012364059686661, -0.12357477843761444, -0.05885322764515877, -0.04981595650315285, -0.05566999688744545, -0.040192216634750366, 0.05430286005139351, -0.05986151844263077, 0.08223418891429901, 0.07280068099498749, 0.11737637966871262, 0.0020225071348249912, 0.06029454246163368, -0.08854582160711288, 0.029932724311947823, 0.004650622606277466, 0.0880812406

In [ ]:
df_news_embedding.count()

15280

In [ ]:
df_news_embedding.select(['hid']).distinct().count()

15280

# Checking for deviceids embeddings using content and title

In [ ]:

path_embedding = "gs://pvtrough_asia_south1/clustering/Content_news_embedding.csv"+sampled_dates[k].strftime("%Y_%m_%d")+"/"


In [ ]:
from pyspark.sql.functions import array, col
path_embedding = 'gs://pvtrough_asia_south1/clustering/Content_news_embedding.csv'
df_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)


# df_embedding.take(1)

In [ ]:
#generate news embeddings based on notification text
newsMapProcessed = {}
document_embeddings_content = {}
document_embeddings={}
embedding_start = time.time()

#try:
#    already_embedded = df_embedding.select('hid').distinct().collect()
#    already_embedded = [x['hid'] for x in already_embedded]
#except:
#    print("News Embeddings not found, generating new one")
already_embedded = []
for hId in tqdm(newsMap):
    
    h = hId.split('-')[0]
#     print(h)
    if h in already_embedded or (h not in hashIdsWithFilter) :
        continue
    else:    
        newsMapProcessed[h] = {}
        newsMapProcessed[h]['title'] = newsMap[hId]['title']
        newsMapProcessed[h]['content'] = newsMap[hId]['content']
        newsMapProcessed[h]['features'] = newsMapProcessed[h]['title'] + newsMapProcessed[h]['content']
        document_embeddings[h] = get_embedding(model, tokenizer, newsMapProcessed[h]['features'])

        document_embeddings_content[h] = document_embeddings[h]

        

        

Log("Embedding took %s minutes"%(int((time.time() - embedding_start) / 60)))
     

 44%|████▍     | 10611/24176 [06:32<10:06, 22.35it/s]

In [286]:
len(document_embeddings_content['l_ebpzzatk'])

768

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [285]:
len(document_embeddings_content)

15280

In [ ]:
#generate news embeddings based on notification text
newsMapProcessed = {}
document_embeddings = {}
try:
    already_embedded = df_embedding.select('hid').distinct().collect()
    already_embedded = [x['hid'] for x in already_embedded]
except:
    print("News Embeddings not found, generating new one")
    already_embedded = []
for hId in tqdm(newsMap):
    
    h = hId.split('-')[0]
#     print(h)
    if h in already_embedded or (h not in hashIdsWithFilter) :
        continue
    else:    
        newsMapProcessed[h] = {}
        newsMapProcessed[h]['title'] = newsMap[hId]['title']
        newsMapProcessed[h]['content'] = newsMap[hId]['content']
        newsMapProcessed[h]['features'] = newsMapProcessed[h]['title'] + "." + newsMapProcessed[h]['content']
        document_embeddings[h] = get_embedding(model, tokenizer, newsMapProcessed[h]['features'])
     

In [287]:
path_embedding = "gs://pvtrough_asia_south1/clustering/Content_news_embedding.csv"+sampled_dates[k].strftime("%Y_%m_%d")+"/"


In [288]:
if len(document_embeddings_content) > 0:
    df_em_pandas = pd.DataFrame(document_embeddings_content).T
    df_em_pandas['hid'] = df_em_pandas.index
    df_em_pandas['embed_date'] = sampled_dates[0]
    df_em_spark = (sqlContext.createDataFrame(df_em_pandas))
    
    df_em_spark.write.partitionBy('embed_date').csv(path_embedding , sep=',', header=True,mode = 'overwrite')

23/06/23 18:46:55 WARN org.apache.spark.scheduler.TaskSetManager: Stage 3398 contains a task of very large size (48558 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:
df_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)

from pyspark.sql.functions import array, col
feat_cols = [str(x) for x in range(0,len(df_embedding.columns) - 2)]
df_embedding = df_embedding.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))
df_embedding.take(1)

#df_embedding=df_embedding.withColumnRenamed("hid","hashId")

In [290]:
df_embedding.count()

15280

In [ ]:
from pyspark.sql.functions import *

k = 0
data_filtered = data_embed
# # calculate idf_weight
N = data_filtered.select('hashId').distinct().count()
users = data_filtered.groupBy('hashId').agg(F.countDistinct('deviceId').alias('users'))
idf_weights = users.withColumn('idf_weight', F.log(N / F.col('users')))

# Multiply embeddings with IDF weights
df_result = df_embedding.join(idf_weights, df_embedding.hid == idf_weights.hashId)
df_result = df_result.withColumn('weighted_embedding', F.expr('transform(embedding, (x, i) -> x * idf_weight)'))


df_result = df_result.select('hid','weighted_embedding')
df_joined = data_filtered.join(df_result, data_filtered.hashId == df_result.hid , 'left')


# assuming your dataframe is called df
grouped_df = df_joined.groupBy("deviceId").agg(*[
    expr(f"avg(weighted_embedding[{i}]) as avg_{i}") for i in range(768)
])



# assuming your dataframe is called grouped_df
avg_cols = [col(f"avg_{i}") for i in range(768)]
grouped_df = grouped_df.withColumn("topic_avg_embedding", array(*avg_cols))

path_vec = "gs://pvtrough_asia_south1/clustering/MLP_user_embeddings"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
path_vec_exploded = "gs://pvtrough_asia_south1/clustering/MLP_device_vectors_exploded_"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
print(path_vec, path_vec_exploded)

grouped_df.select('deviceId', 'topic_avg_embedding').write.parquet(path=path_vec, mode='overwrite')
#grouped_df.select('deviceId', *avg_cols).write.parquet(path=path_vec_exploded, mode='overwrite')


23/06/23 19:16:38 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 2.0 MiB


In [ ]:
grouped_df.take(1)

In [293]:
from pyspark.sql.functions import array_contains, size, col
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

k = 0
#num_clusters = 1000
path_vec = "gs://pvtrough_asia_south1/clustering/MLP_user_embeddings"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
# path_vec = "gs://pvtrough_asia_south1/clustering/device_vectors_roberta_live/"
print(path_vec)
# print(path_vec)

temp = sqlContext.read.parquet(path_vec)
df_devices_embeddings = temp.filter(~array_contains(temp.topic_avg_embedding, float('nan')))


df_devices_embeddings = df_devices_embeddings.dropna(how='all')
df_devices_embeddings = df_devices_embeddings.where(size(col("topic_avg_embedding")) == 768)







gs://pvtrough_asia_south1/clustering/MLP_user_embeddings2023_02_01/


In [ ]:
df_devices_embeddings.take(1)

# Reducing the size of vectors using LDA for device and news embeddings

In [311]:
df_news_embedding = df_news_embedding.where(size(col("embedding")) == 768)


In [313]:
train_data = data_train.join(df_devices_embeddings,["deviceId"],"inner")
train_data = train_data.join(df_news_embedding,train_data.hashId == df_news_embedding.hid,"inner")
train_data = train_data.select('deviceId', 'hashId', 'topic_avg_embedding', 'embedding','notiOpened')
train_data = train_data.withColumn("notiOpened",F.when(train_data.notiOpened > 1, 
                                                     1).otherwise(train_data.notiOpened))
train_data.printSchema()

root
 |-- deviceId: string (nullable = true)
 |-- hashId: string (nullable = true)
 |-- topic_avg_embedding: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- embedding: array (nullable = false)
 |    |-- element: double (containsNull = true)
 |-- notiOpened: long (nullable = true)



In [ ]:
from pyspark.sql.functions import countDistinct

train_data.groupby('notiOpened').count().show()

In [ ]:
embedding_length = 256
train_data = train_data.dropna(how='all')
train_data = train_data.where((size(col("topic_avg_embedding")) == embedding_length) & (size(col("embedding")) == embedding_length))

# train_data.count()

In [315]:
from pyspark.sql.functions import col, concat

train_data_features = train_data.withColumn('embedding_features', 
                                            concat(col('topic_avg_embedding'), col('embedding')))\
                                .select('embedding_features', 'notiOpened')
                                    

In [ ]:
train_data_features.take(1)

In [ ]:
jhuh=[0.0023, 0.0477, -0.0396, 0.0009, 0.0234, -0.0007, -0.0263, 0.0094, -0.003, -0.0079, 0.0083, 0.0157, 0.0157, 0.0268, -0.0272, 0.016, -0.0234, -0.0126, -0.0291, -0.0232, -0.0609, -0.0379, -0.0736, 0.0412, 0.0053, -0.0663, 0.0372, 0.0556, 0.0942, 0.0222, -0.0089, -0.0569, -0.085, 0.073, -0.0409, 0.0562, 0.0087, 0.0767, 0.0242, 0.1207, 0.3078, -0.0326, -0.0746, 0.0526, -0.0916, -0.1128, 0.0008, -0.1823, -0.0065, 0.0489, 0.001, -0.0053, -0.0069, -0.0497, 0.0191, -0.063, 0.0224, -0.0335, -0.0016, 0.0158, -0.0006, 0.0125, -0.007, 0.0115, -0.0181, -0.0065, -0.0081, -0.0752, 0.0297, 0.0259, 0.0103, 0.1363, 0.0398, 0.0333, -0.0134, 0.0258, -0.0841, 0.1014, 0.0526, -0.1862, -0.0682, 0.16, -0.0917, -0.0708, -0.4389, 0.1636, -0.0263, -0.2365, -0.2017, -0.1257, -0.2023, -0.4874, 0.1562, 0.4067, -0.0379, -0.0119, 0.1549, -0.1315, 0.1141, 0.076]

In [ ]:
len(jhuh)

In [318]:
from pyspark.ml.functions import array_to_vector

train_data_features=train_data_features.select(array_to_vector('embedding_features').alias('vec_embedding'),'notiOpened')#.collect()

In [ ]:
train_data_features.take(1)

In [ ]:
train_data_features.count()

In [ ]:
d1 = data_train.select('deviceId').distinct().collect()
d2 = train_data.select('deviceId').distinct().collect()

In [ ]:
left_out = set(d1) - set(d2)
len(left_out)

In [ ]:
merged_df = train_data.sample(fraction = 0.01)

In [ ]:
assembler = VectorAssembler(inputCols = ['embedding_features'], outputCol = 'vector_features')
train_data_vector = assembler.transform(train_data_features)

# using underampling for imbalanced classes

In [320]:
train_data_features

DataFrame[vec_embedding: vector, notiOpened: bigint]

In [ ]:
major_df = train_data_features.filter(col("notiOpened") == 0)
minor_df = train_data_features.filter(col("notiOpened") == 1)
ratio = int(major_df.count()/minor_df.count())

In [323]:
print("ratio: {}".format(ratio))


ratio: 14


In [324]:
sampled_majority_df = major_df.sample(False, 1/ratio)
combined_df_2 = sampled_majority_df.unionAll(minor_df)
#combined_df_2.show()

In [325]:
combined_df_2.count()# 83031423

23/06/23 19:50:26 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1022.2 KiB
23/06/23 19:50:38 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1053.7 KiB
23/06/23 19:51:51 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1339.2 KiB
Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


83031423

In [ ]:
combined_df_2.unpersist()

In [326]:
combined_df_2

DataFrame[vec_embedding: vector, notiOpened: bigint]

In [ ]:
combined_df_2.cache()

In [327]:
path_vec=path_vec = "gs://pvtrough_asia_south1/clustering/MLP_undersampled_train"+sampled_dates[k].strftime("%Y_%m_%d")+"/"

combined_df_2.select('vec_embedding', 'notiOpened').write.parquet(path=path_vec, mode='overwrite')


23/06/23 19:53:10 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1566.0 KiB


In [ ]:

training_start = time.time()

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import expr, udf
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


data = combined_df_2.select("vec_embedding", "notiOpened").sample(fraction = 0.01)

# Split the data into training and testing sets
(training_data, testing_data) = data.randomSplit([0.7, 0.3])#.sample(fraction)

# Define the deep learning model architecture
layers = [1536,1400,1000,600,450,300,150,50,10,2]  # Adjust the layer sizes as needed
classifier = MultilayerPerceptronClassifier(layers=layers, labelCol="notiOpened", featuresCol="vec_embedding",solver='gd')
# add loss function to check training ho rha hai ya nhi
# Train the model
model_class = classifier.fit(training_data)
result = model_class.transform(testing_data)
# # Make predictions on the testing data
# predictions = model.transform(testing_data)

# # Evaluate the model
# evaluator = BinaryClassificationEvaluator(labelCol="notiOpened", rawPredictionCol="prediction")
# areaUnderROC = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
# print("Area under ROC curve:", areaUnderROC)
Log("Model fitting took %s minutes"%(int((time.time() - training_start) / 60)))


In [ ]:
# add loss function to check training ho rha hai ya nhi, 
# reduce layers size to half
# dimension reduction using PCA

In [ ]:
y_true = result.select(['notiOpened']).collect()
y_pred = result.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
Log(classification_report(y_true, y_pred))


In [ ]:
preds_and_labels = result.select(['prediction','notiOpened']).withColumn('label', F.col('notiOpened').cast(DoubleType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','label'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
cm=metrics.confusionMatrix().toArray()
Log(cm)


In [335]:
accuracy=(cm[0][0]+cm[1][1])/cm.sum()
precision=(cm[1][1])/(cm[0][1]+cm[1][1])
recall=(cm[1][1])/(cm[1][0]+cm[1][1])
f1_score=(2*precision*recall)/(recall+precision)
print(accuracy, precision, recall,f1_score)

0.5005877822368154 nan 0.0 nan


/tmp/ipykernel_24395/3738225083.py:2: RuntimeWarning: invalid value encountered in double_scalars
  precision=(cm[1][1])/(cm[0][1]+cm[1][1])


In [ ]:
# create classifier on very high precision on a) for opening b) for not opening c) with imbalance data

# Trying logistic regression

In [ ]:
# next two blocks for class weight
y_collect = train_data_features.select("notiOpened").groupBy("notiOpened").count().collect()
unique_y = [x["notiOpened"] for x in y_collect]
total_y = sum([x["count"] for x in y_collect])
unique_y_count = len(y_collect)
bin_count = [x["count"] for x in y_collect]

class_weights_spark = {i: ii for i, ii in zip(unique_y, total_y / (unique_y_count * np.array(bin_count)))}
print(class_weights_spark)

In [ ]:
mapping_expr = F.create_map([F.lit(x) for x in chain(*class_weights_spark.items())])

train_data_features = train_data_features.withColumn("weight", mapping_expr.getItem(F.col("y")))


In [ ]:
training_start = time.time()
from pyspark.ml.classification import LogisticRegression
data = combined_df_2.select("vec_embedding", "notiOpened")#.sample(fraction = 0.8)

# Split the data into training and testing sets
(training_data, testing_data) = data.randomSplit([0.7, 0.3])
lr = LogisticRegression(featuresCol="vec_embedding", labelCol="notiOpened")
lrModel = lr.fit(training_data)
result = lrModel.transform(testing_data)
Log("Model training took %s minutes"%(int((time.time() - training_start) / 60)))



In [ ]:
y_true = result.select(['notiOpened']).collect()
y_pred = result.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
Log(classification_report(y_true, y_pred))


In [341]:
# sample of 0.01
preds_and_labels = result.select(['prediction','notiOpened']).withColumn('label', F.col('notiOpened').cast(DoubleType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','label'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
cm=metrics.confusionMatrix().toArray()
Log(cm)


[[74910. 51053.]
 [50202. 72411.]]


In [342]:
#sample of 0.01
accuracy=(cm[0][0]+cm[1][1])/cm.sum()
precision=(cm[1][1])/(cm[0][1]+cm[1][1])
recall=(cm[1][1])/(cm[1][0]+cm[1][1])
f1_score=(2*precision*recall)/(recall+precision)
print(accuracy, precision, recall,f1_score)

0.5926597901647785 0.5864948487008359 0.5905654375963397 0.5885231045567038


In [345]:
#complete data
preds_and_labels = result.select(['prediction','notiOpened']).withColumn('label', F.col('notiOpened').cast(DoubleType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','label'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
cm=metrics.confusionMatrix().toArray()
Log(cm)


[[7604563. 5016202.]
 [5065028. 7222488.]]


In [346]:
#complete data
accuracy=(cm[0][0]+cm[1][1])/cm.sum()
precision=(cm[1][1])/(cm[0][1]+cm[1][1])
recall=(cm[1][1])/(cm[1][0]+cm[1][1])
f1_score=(2*precision*recall)/(recall+precision)
print(accuracy, precision, recall,f1_score)

0.5952659278253686 0.590135708968852 0.5877907300385204 0.5889608853485125


# on training data 

In [ ]:
result_training = lrModel.transform(training_data)
y_true = result_training.select(['notiOpened']).collect()
y_pred = result_training.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
Log(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.60      0.60      0.60  29454134
           1       0.59      0.59      0.59  28669008

    accuracy                           0.60  58123142
   macro avg       0.60      0.60      0.60  58123142
weighted avg       0.60      0.60      0.60  58123142



In [ ]:

preds_and_labels = result_training.select(['prediction','notiOpened']).withColumn('label', F.col('notiOpened').cast(DoubleType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','label'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
cm=metrics.confusionMatrix().toArray()
Log(cm)


[[17752136. 11701998.]
 [11818570. 16850438.]]


In [ ]:
#sample of 0.01
accuracy=(cm[0][0]+cm[1][1])/cm.sum()
precision=(cm[1][1])/(cm[0][1]+cm[1][1])
recall=(cm[1][1])/(cm[1][0]+cm[1][1])
f1_score=(2*precision*recall)/(recall+precision)
print(accuracy, precision, recall,f1_score)

0.5953321312189214 0.5901576313838861 0.5877579719535465 0.5889553573656757


# Training on 10 days and predicting on 7 days data

In [351]:
path_embedding = "gs://pvtrough_asia_south1/clustering/Content_news_embedding.csv"+sampled_dates[k].strftime("%Y_%m_%d")+"/"


In [352]:
df_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)

from pyspark.sql.functions import array, col
feat_cols = [str(x) for x in range(0,len(df_embedding.columns) - 2)]
df_embedding = df_embedding.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))
df_embedding.take(1)

#df_embedding=df_embedding.withColumnRenamed("hid","hashId")

[Row(hid='svovy3mz', embedding=[-0.026875313371419907, 0.1945246458053589, 0.02433736063539982, -0.09676317125558853, 0.16876408457756042, 0.006928965449333191, 0.01006230991333723, 0.04343455657362938, 0.001783062587492168, -0.05376378819346428, -0.03129350394010544, -0.11458952724933624, 0.05109705775976181, -0.10799243301153183, 0.02900136634707451, -0.02166307531297207, 0.1632353961467743, -0.02483336813747883, -0.08497937023639679, 0.25693243741989136, 0.028930554166436195, 0.08322273194789886, -0.14118240773677826, -0.162668839097023, -0.010364331305027008, -0.020930470898747444, 0.06786119937896729, 0.1229231059551239, -0.05521037429571152, 0.016381191089749336, 0.006578350905328989, -0.07388663291931152, 0.09268548339605331, 0.09028810262680054, -0.043422721326351166, 0.05357334017753601, 0.07575681805610657, 0.03570711612701416, 0.3640774190425873, -0.007589297369122505, -0.04624001309275627, 0.0024187632370740175, -0.09080038219690323, 0.002034494187682867, -0.033694427460432

In [ ]:
from pyspark.sql.functions import *

k = 0
data_filtered = data_comb
# # calculate idf_weight
N = data_filtered.select('hashId').distinct().count()
users = data_filtered.groupBy('hashId').agg(F.countDistinct('deviceId').alias('users'))
idf_weights = users.withColumn('idf_weight', F.log(N / F.col('users')))

# Multiply embeddings with IDF weights
df_result = df_embedding.join(idf_weights, df_embedding.hid == idf_weights.hashId)
df_result = df_result.withColumn('weighted_embedding', F.expr('transform(embedding, (x, i) -> x * idf_weight)'))


df_result = df_result.select('hid','weighted_embedding')
df_joined = data_filtered.join(df_result, data_filtered.hashId == df_result.hid , 'left')


# assuming your dataframe is called df
grouped_df = df_joined.groupBy("deviceId").agg(*[
    expr(f"avg(weighted_embedding[{i}]) as avg_{i}") for i in range(768)
])



# assuming your dataframe is called grouped_df
avg_cols = [col(f"avg_{i}") for i in range(768)]
grouped_df = grouped_df.withColumn("topic_avg_embedding", array(*avg_cols))

path_vec = "gs://pvtrough_asia_south1/clustering/MLP_user_embeddings40"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
path_vec_exploded = "gs://pvtrough_asia_south1/clustering/MLP_device_vectors_exploded_40"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
print(path_vec, path_vec_exploded)

grouped_df.select('deviceId', 'topic_avg_embedding').write.parquet(path=path_vec, mode='overwrite')
#grouped_df.select('deviceId', *avg_cols).write.parquet(path=path_vec_exploded, mode='overwrite')


In [356]:
from pyspark.sql.functions import array_contains, size, col
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

k = 0
#num_clusters = 1000
path_vec = "gs://pvtrough_asia_south1/clustering/MLP_user_embeddings40"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
# path_vec = "gs://pvtrough_asia_south1/clustering/device_vectors_roberta_live/"
print(path_vec)
# print(path_vec)

temp = sqlContext.read.parquet(path_vec)
df_devices_embeddings = temp.filter(~array_contains(temp.topic_avg_embedding, float('nan')))


df_devices_embeddings = df_devices_embeddings.dropna(how='all')
df_devices_embeddings = df_devices_embeddings.where(size(col("topic_avg_embedding")) == 768)







gs://pvtrough_asia_south1/clustering/MLP_user_embeddings402023_02_01/


In [358]:
df_news_embedding

DataFrame[hid: string, embedding: array<double>]

In [360]:
test_data = data_test.join(df_devices_embeddings,["deviceId"],"inner")
test_data = test_data.join(df_news_embedding,test_data.hashId == df_news_embedding.hid,"inner")
test_data = test_data.select('deviceId', 'hashId', 'topic_avg_embedding', 'embedding','notiOpened')
test_data = test_data.withColumn("notiOpened",F.when(test_data.notiOpened > 1, 
                                                     1).otherwise(test_data.notiOpened))
test_data.printSchema()

root
 |-- deviceId: string (nullable = true)
 |-- hashId: string (nullable = true)
 |-- topic_avg_embedding: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- embedding: array (nullable = false)
 |    |-- element: double (containsNull = true)
 |-- notiOpened: long (nullable = true)



In [361]:
from pyspark.sql.functions import col, concat

test_data_features = test_data.withColumn('embedding_features', 
                                            concat(col('topic_avg_embedding'), col('embedding')))\
                                .select('embedding_features', 'notiOpened')
                                    

In [362]:
from pyspark.ml.functions import array_to_vector

test_data_features=test_data_features.select(array_to_vector('embedding_features').alias('vec_embedding'),'notiOpened')#.collect()

In [ ]:
path_vec = "gs://pvtrough_asia_south1/clustering/test_data"+sampled_dates[k].strftime("%Y_%m_%d")+"/"

test_data_features.select('vec_embedding', 'notiOpened').write.parquet(path=path_vec, mode='overwrite')


In [ ]:
Log("training started")


testing_data=test_data_features

#test data
result = lrModel.transform(testing_data)

y_true = result.select(['notiOpened']).collect()
y_pred = result.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
Log("Classification report test data")

Log(classification_report(y_true, y_pred))




preds_and_labels = result.select(['prediction','notiOpened']).withColumn('label', F.col('notiOpened').cast(DoubleType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','label'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
cm=metrics.confusionMatrix().toArray()
Log("confusion matrix test data")

Log(cm)

#sample of 0.01
accuracy=(cm[0][0]+cm[1][1])/cm.sum()
precision=(cm[1][1])/(cm[0][1]+cm[1][1])
recall=(cm[1][1])/(cm[1][0]+cm[1][1])
f1_score=(2*precision*recall)/(recall+precision)
print(accuracy, precision, recall,f1_score)
Log(" accuracy, precision, recall, f1_score for test data")
Log(" accuracy on test data = %g" % accuracy)
Log(" precision on test data = %g" % precision)
Log(" recall on test data = %g" % recall)
Log(" f1_score on test data = %g" % f1_score)


Log("Model training took %s minutes"%(int((time.time() - training_start) / 60)))


In [ ]:
training_start = time.time()
from pyspark.ml.classification import LogisticRegression
training_data = combined_df_2.select("vec_embedding", "notiOpened")#.sample(fraction = 0.8)

# Split the data into training and testing sets
#(training_data, testing_data) = data.randomSplit([0.7, 0.3])
testing_data=test_data_features
lr = LogisticRegression(featuresCol="vec_embedding", labelCol="notiOpened")
lrModel = lr.fit(training_data)

result_training = lrModel.transform(training_data)
y_true = result_training.select(['notiOpened']).collect()
y_pred = result_training.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
Log("Classification report training data")

Log(classification_report(y_true, y_pred))




preds_and_labels = result_training.select(['prediction','notiOpened']).withColumn('label', F.col('notiOpened').cast(DoubleType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','label'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
cm=metrics.confusionMatrix().toArray()
Log("confusion matrix training data")

Log(cm)

#sample of 0.01
accuracy=(cm[0][0]+cm[1][1])/cm.sum()
precision=(cm[1][1])/(cm[0][1]+cm[1][1])
recall=(cm[1][1])/(cm[1][0]+cm[1][1])
f1_score=(2*precision*recall)/(recall+precision)
print(accuracy, precision, recall,f1_score)
Log(" accuracy, precision, recall, f1_score for traininga data")
Log(" accuracy on train data = %g" % accuracy)
Log(" precision on train data = %g" % precision)
Log(" recall on train data = %g" % recall)
Log(" f1_score on train data = %g" % f1_score)

# adding category feature also in the data 

In [ ]:
from pyspark.sql.types import DoubleType

predictionAndLabels = result.select(['prediction', 'notiOpened']\
                                  ).withColumn('label',F.col('notiOpened').cast(DoubleType())).rdd

metrics = MulticlassMetrics(predictionAndLabels)

cm = metrics.confusionMatrix().toArray()

accuracy=(cm[0][0]+cm[1][1])/cm.sum()
precision=(cm[1][1])/(cm[0][1]+cm[1][1])
recall=(cm[1][1])/(cm[1][0]+cm[1][1])

print(accuracy, precision, recall)

In [ ]:
model_class

In [ ]:
result = model_class.transform(training_data)
import pyspark.sql.functions as F

In [ ]:
testing_data.printSchema()

In [ ]:
import pyspark.sql.functions as F
testing_data.filter(F.size(F.col('vec_embedding'))!=50).count()

In [ ]:
import pyspark.sql.functions as F

In [ ]:
result.columns

In [ ]:
result.cache()

In [ ]:
result.take(1)

# Random Forest

In [ ]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
training_start = time.time()

data = combined_df_2.select("vec_embedding", "notiOpened")#.sample(fraction = 0.1)
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])


model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

predictions = model.predict(testData.map(lambda x: x.vec_embedding))
labelsAndPredictions = testData.map(lambda lp: lp.notiOpened).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())

# Save and load model
#model.save(sc, "target/tmp/myRandomForestClassificationModel")
#sameModel = RandomForestModel.load(sc, "target/tmp/myRandomForestClassificationModel")
Log("Model fitting took %s minutes"%(int((time.time() - training_start) / 60)))


In [ ]:
y_true = result.select(['notiOpened']).collect()
y_pred = result.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

# Model evaluation

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictionAndLabels = result.select("prediction", "notiOpened")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(predictionAndLabels)

# Overall statistics
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)

In [ ]:
preds_and_labels = result.select(['prediction','notiOpened']).withColumn('label', F.col('notiOpened').cast(DoubleType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','label'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))


In [210]:
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)

TypeError: precision() missing 1 required positional argument: 'label'

In [207]:
print(metrics.confusionMatrix().toArray())


[[2.1084e+04 1.8000e+01]
 [2.0385e+04 2.0000e+00]]


In [214]:
2.0385e+04/(2.1084e+04+2.0385e+04)

0.49157201765173986

In [213]:
2.0385e+04/(2.0385e+04+2.0000e+00)

0.9991470002843332

In [ ]:
result.count()

In [ ]:
result.select(['notiOpened','prediction']).show(40)

In [ ]:
result

In [211]:
y_true = result.select(['notiOpened']).collect()
y_pred = result.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.51      1.00      0.67     21102
           1       0.20      0.00      0.00     20387

    accuracy                           0.51     41489
   macro avg       0.35      0.50      0.34     41489
weighted avg       0.36      0.51      0.34     41489



In [ ]:
#select only prediction and label columns
from pyspark.mllib.evaluation import MulticlassMetrics

preds_and_labels = result.select(['prediction','notiOpened'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())
Log(metrics.confusionMatrix().toArray())


In [ ]:
Log(metrics.confusionMatrix().toArray())


In [ ]:
training_data

In [ ]:
from pymongo import UpdateOne
import datetime
import logging
import os
import shutil
import sys
import json

import numpy as np
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import *
from Utils import *
from tqdm import tqdm

mongoClient = getMongoClient()
trainDB = mongoClient['trialDB_roberta']
deviceVectorCollection = trainDB['deviceVectors']
deviceClusterCollection = trainDB['notiDeviceClusters']

newsVectorCollection = trainDB['newsVectors']
newsTSpentCollection = trainDB['newsTSpent']
newsScoreCollection = trainDB['newsScores']
deviceNotiClustersCollection = trainDB['notiDeviceClusters']

clusterCentersCollection = trainDB['clusterCenters']
 
def getDeviceVectorsFromMongo(deviceIds):
    
    deviceVectorMap = {}
    cursor = deviceVectorCollection.find({"_id" : { "$in" : deviceIds }})
    
    for c in cursor:
        deviceVectorMap[c['_id']] = np.array(c['vector'])
    
    return deviceVectorMap
        
def getNewsVectorsFromMongo(hashIds):
    newsVectorMap = {}
    cursor = newsVectorCollection.find({"_id" : { "$in" : hashIds }})
    
    for c in cursor:
        newsVectorMap[c['_id']] = np.array(c['vector'])
    
    return newsVectorMap


def insertDeviceVectorsInMongo(deviceVectors):
    
    collection = deviceVectorCollection
    ops = []
    results = []
    for deviceId in tqdm(deviceVectors):
        key = {"_id" : deviceId}
        devData = {"$set" : {"vector" : deviceVectors[deviceId]}}
        ops.append(UpdateOne(key, devData, upsert=True))
    
        if len(ops) == 100000:
            results.append(collection.bulk_write(ops,ordered=False))
            ops = []

    if len(ops) > 0:
        results.append(collection.bulk_write(ops,ordered=False))

def insertNewsVectorsInMongo(newsVectors):
    for hashId in newsVectors:
        key = {"_id" : hashId}
        newsData = {"$set" : {"vector" : list(newsVectors[hashId])}}
        newsVectorCollection.update_one(key, newsData, upsert=True)
        
        
oldClusters = {}
def getDeviceClustersFromMongo(deviceIds):
    print("here: ", deviceClusterCollection)
    deviceClusterMap = {}
    if len(deviceIds) == 0:
        cursor = deviceClusterCollection.find()
    else:
        cursor = deviceClusterCollection.find({"_id" : { "$in" : deviceIds }})
    
    for c in cursor:
        deviceClusterMap[c['_id']] = c['cluster']
        oldClusters[c['_id']] = c['cluster']
    
    return deviceClusterMap

"""deprecated"""
# def insertDeviceClustersInMongo(deviceClusters):
#     collection = deviceClusterCollection
#     ops = []
#     results = []
    
#     for deviceId in deviceClusters:
#         key = {"_id" : deviceId}
        
#         devData = {"$set" : {"cluster" : int(deviceClusters[deviceId])}}
        
#         if deviceId not in oldClusters or int(deviceClusters[deviceId]) != oldClusters[deviceId]:
#             devData = {"$set" : {"cluster" : int(deviceClusters[deviceId]), "updatedAt" : datetime.datetime.now()}}
            
#         ops.append(UpdateOne(key, devData, upsert=True))
    
#         if len(ops) == 1000:
#             results.append(collection.bulk_write(ops,ordered=False))
#             ops = []

#     if len(ops) > 0:
#         results.append(collection.bulk_write(ops,ordered=False))
        


def insertNewsVectorsInMongo(newsVectors):
    for hashId in newsVectors:
        key = {"_id" : hashId}
        newsData = {"$set" : {"vector" : list(newsVectors[hashId])}}
        newsVectorCollection.update_one(key, newsData, upsert=True)
        
def getNewsTSpentFromMongo(hashIds):
    newsTSpentMap = {}
    cursor = newsTSpentCollection.find({"_id" : {"$in" : hashIds }})
    
    for c in cursor:
        data = {}
        for cluster in c['tSpent']:
            data[int(cluster)] = c['tSpent'][cluster]
        newsTSpentMap[c['_id']] = data
        
    return newsTSpentMap

def insertNewsTSpentInMongo(newsTSpentMap):
    for hashId in newsTSpentMap:
        key = {"_id" : hashId}
        data = {}
        for cluster in newsTSpentMap[hashId]:
            data[str(cluster)] = newsTSpentMap[hashId][cluster]
        tSpentData = {"$set" : {"tSpent" : data}}
        newsTSpentCollection.update_one(key, tSpentData, upsert=True)

def getNewsScoreInMongo(hashIds):
    newsScoreMap = {}
    cursor = newsScoreCollection.find({"_id" : {"$in" : hashIds }})
    
    for c in cursor:
        data = {}
        for cluster in c['tSpent']:
            data[int(cluster)] = c['tSpent'][cluster]
        newsScoreMap[c['_id']] = data
        
    return newsScoreMap

def insertNewsScoreInMongo(newsTSpentMap):
    for hashId in newsTSpentMap:
        key = {"_id" : hashId}
        data = {}
        for cluster in newsTSpentMap[hashId]:
            if newsTSpentMap[hashId][cluster][1] > 1:
                data[str(cluster)] = newsTSpentMap[hashId][cluster][0] / newsTSpentMap[hashId][cluster][1]  
        
        scoreData = {"$set" : {"tSpent" : data, "updatedAt" : datetime.datetime.now()}}
        newsScoreCollection.update_one(key, scoreData, upsert=True)

def insertDeviceClustersInMongo(deviceClusters, oldClusters, Log):
    ops = []
    #results = []
    
    cntUpdated = 0
    for deviceId in deviceClusters:
        key = {"_id" : deviceId}
        
        devData = {"$set" : {"cluster" : int(deviceClusters[deviceId]), "updatedAt" : datetime.datetime.now()}}
        
        if deviceId in oldClusters and deviceClusters[deviceId] == oldClusters[deviceId]:
            continue
        
        cntUpdated += 1
        ops.append(UpdateOne(key, devData, upsert=True))
    
        if len(ops) == 100000:
            try:
                result = deviceNotiClustersCollection.bulk_write(ops,ordered=False)
                ops = []
            except Exception as e:
                ops = []
                Log("Failed to insert devices clusters, " + str(e))
        
        if cntUpdated % 100000 == 0:
            Log("Inserted " + str(cntUpdated) + " Device Clusters")
                
    if len(ops) > 0:
        deviceNotiClustersCollection.bulk_write(ops,ordered=False)
    
    print(cntUpdated,len(deviceClusters))
    Log("Updated  device clusters")


In [ ]:

# from ColabUtils import *
device_vect_df = df.toPandas()
device_vect_df

deviceVectors = {}
for v in tqdm(device_vect_df.values):
    deviceVectors[v[0]] = v[1]
insertDeviceVectorsInMongo(deviceVectors)

In [ ]:
labels = dfdump.select('deviceId','cluster').toPandas()
labels = pd.Series(labels.cluster.values,index=labels.deviceId).to_dict()
d4 = labels.keys()

In [ ]:

newClusterWiseDevices = defaultdict(lambda: set())
for deviceId, cluster in tqdm(labels.items()):
    newClusterWiseDevices[cluster].add(deviceId)

In [ ]:
# from ColabUtils import *
deviceClusterMapExisiting = getDeviceClustersFromMongo([])

In [ ]:
clusterWiseDevices = defaultdict(lambda: set())
for d in tqdm(deviceClusterMapExisiting):
    clusterWiseDevices[deviceClusterMapExisiting[d]].add(d)

In [ ]:

marked = {}
clusterMap = {}
for x in newClusterWiseDevices:
    mxinter = -1
    mxidx = -1
    for y in clusterWiseDevices:
        if y in marked:
            continue
        inter = (len(clusterWiseDevices[y].intersection(newClusterWiseDevices[x])) / len(clusterWiseDevices[y])*100)
        if inter > mxinter:
            mxinter = inter
            mxidx = y
            
    if mxidx < 0:
        mxidx = x
    
    marked[mxidx] = 1
    clusterMap[x] = mxidx
    Log("Mapping cluster " + str(x) + " to " + str(mxidx) + ", %age intersection = " + str(mxinter))

In [ ]:
updatedDeviceClusters = {}
for cluster in newClusterWiseDevices:
    for deviceId in tqdm(newClusterWiseDevices[cluster]):
        updatedDeviceClusters[deviceId] = clusterMap[cluster]

In [ ]:
insert_start = time.time()
# from ColabUtils import *
insertDeviceClustersInMongo(updatedDeviceClusters, deviceClusterMapExisiting, Log)

In [ ]:
insert_time = (time.time() - insert_start)

Log("Insertion took %s percent of time" % (100 * insert_time / (time.time() - start_time)))

In [ ]:
dataWithClusters = data_filtered.select('hashId', 'overallTimeSpent', F.udf(lambda deviceId: updatedDeviceClusters[deviceId], IntegerType())('deviceId').alias('cluster'))
newsClusterMean = dataWithClusters.groupby('hashId', 'cluster').agg(F.sum('overallTimeSpent'),F.count('hashId'))
newsClusterMeanDf = newsClusterMean.toPandas()

In [ ]:
newsTSpentMap = defaultdict(lambda: defaultdict(lambda: (7, 1)))
for v in tqdm(newsClusterMeanDf.values):
    newsTSpentMap[v[0]][v[1]] = (v[2], v[3])

In [ ]:
Log("Inserting News Score Data")
insert_start = time.time()
insertNewsTSpentInMongo(newsTSpentMap)
insertNewsScoreInMongo(newsTSpentMap)
insert_time = (time.time() - insert_start)
Log("Inserting News Score took %s percent of time" % str(100 * insert_time / (time.time() - start_time)))

In [ ]:
def get_raw_path_notification(date, hours=None):
    paths = []
    base_path = NIS_RAW_DATA_BASE_PATH + date
    if not hours:
        return base_path + "/*/*.gz"
    for hour in hours:
        paths.append(base_path + "/" + str(hour).zfill(2) + "/*.gz")
    return ",".join(paths)

def process_raw_data_notification(paths):
    def view_data_filters(x):
        x = x['properties']
        deviceid_filter = ('device_id' in x) and (x['device_id'] != '')
        noti_filter = ('notiOpened' in x) and (int(x['timeSpent']) <= 100)
        return deviceid_filter and time_filter

    try:
        rdd = sc.textFile(paths)             .map(json.loads)   .filter(lambda x: ("event_name" in x) and ( (x["event_name"].lower() == "notification opened") or (x["event_name"].lower() == "notification shown") ))            
        view_data = rdd.map(lambda x: (x['device_id'], x['hashId'][:-2], x['short_time'], x['noti_opened'], x['noti_shown'], x['event_name']))             .toDF(['deviceId', 'hashId', 'timeSpent', 'notiOpened', 'notiShown', 'event_name'])
        
        view_data = view_data.withColumn("notiOpened",F.when(view_data.event_name == 'Notification Opened', 1).otherwise(0))
        view_data = view_data.withColumn("notiShown",F.when(view_data.event_name == 'Notification Shown', 1).otherwise(0))
        view_data = view_data.select('deviceId', 'hashId', 'timeSpent', 'notiOpened', 'notiShown')  .groupby(view_data.deviceId, view_data.hashId)                              .agg(F.max(view_data.timeSpent).alias('timeSpent'),F.sum(view_data.notiOpened).alias('notiOpened'),F.sum(view_data.notiShown).alias('notiShown'))
        view_data = view_data.withColumn("notiShown",F.when(view_data.notiShown < view_data.notiOpened, 
                                                     view_data.notiOpened).otherwise(view_data.notiShown))
        return view_data
    except Exception as e:
        logger.warning("Error processing data: " + str(e))


In [ ]:
NIS_RAW_DATA_BASE_PATH = "gs://inshorts-minimal-event/data/inshorts-minimal-event-v1/"

path = get_raw_path_notification(datetime.datetime.now().strftime("%Y/%m/%d"))
today_data = process_raw_data_notification(path)
today_data = today_data.filter(today_data.hashId.isin(hashIdsWithFilter))

In [ ]:
n_days = 7
st_date = datetime.datetime.now() - datetime.timedelta(days=n_days+1)

date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]

# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

hashIdsWithFilter, newsMap = getNewsData(dates[0], datetime.datetime.now())

In [ ]:
datestr = [d.strftime(date_fmt) for d in dates]
paths = get_path(datestr, 'otherEvents')

data = sqlContext.read.parquet(*paths)

data = filter_app(data, app_name=None)
data = filter_tenant(data, tenant='en')

# data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))

data = data.select(data.deviceId, (F.split(data.hashId, '-')[0]).alias('hashId'), data.overallTimeSpent, data.notificationOpened, data.notificationShown, data.notificationType,data.eventName)
data = data.filter( data.eventName.isin(['Notification Shown','Notification Opened']) )

data = data.filter(data.hashId.isin(hashIdsWithFilter))

In [ ]:
data = data.select('deviceId', 'hashId' ,'overallTimeSpent', 'notificationOpened', 'notificationShown')\
            .groupby('deviceId', 'hashId')\
            .agg(F.max('overallTimeSpent').alias('timeSpent'), 
                 F.sum('notificationShown').alias('notiShown'),F.sum('notificationOpened').alias('notiOpened'))
data = data.withColumn("notiShown",F.when(data.notiShown < data.notiOpened, 
                                                     data.notiOpened).otherwise(data.notiShown))

In [ ]:
data = data.union(today_data)
data.cache()

In [ ]:

df_temp1=data_filtered.select(['deviceid']).distinct()
df_temp1=df_temp.withColumnRenamed("deviceid","deviceId")

data=data.join(df_temp1,["deviceId"],"inner")

In [ ]:
dataWithClusters = data.select('hashId', 'notiOpened','notiShown', F.udf(lambda deviceId: updatedDeviceClusters[deviceId], IntegerType())('deviceId').alias('cluster'))
newsClusterNotiMean = dataWithClusters.groupby('hashId', 'cluster').agg(F.sum('notiOpened'),F.sum('notiShown'),F.count('hashId'))
# newsClusterNotiMeanDf = newsClusterNotiMean.toPandas()

In [ ]:
newsClusterNotiMean.show()

In [ ]:
newsNotiSpentCollection = trainDB['newsNotiSpent']
newsNotiScoreCollection = trainDB['newsNotiScores']


def getNewsNotiSpentFromMongo(hashIds):
    newsNotiSpentMap = {}
    cursor = newsNotiSpentCollection.find({"_id" : {"$in" : hashIds }})
    
    for c in cursor:
        data = {}
        for cluster in c['nOpened']:
            data[int(cluster)] = c['nOpened'][cluster]
        newsNotiSpentMap[c['_id']] = data
        
    return newsNotiSpentMap

def insertNewsNotiSpentInMongo(newsNotiSpentMap):
    for hashId in newsNotiSpentMap:
        key = {"_id" : hashId}
        data = {}
        for cluster in newsNotiSpentMap[hashId]:
            data[str(cluster)] = newsNotiSpentMap[hashId][cluster]
        NotiSpentData = {"$set" : {"nOpened" : data}}
        newsNotiSpentCollection.update_one(key, NotiSpentData, upsert=True)

def getNewsNotiScoreInMongo(hashIds):
    newsNotiScoreMap = {}
    cursor = newsNotiScoreCollection.find({"_id" : {"$in" : hashIds }})
    
    for c in cursor:
        data = {}
        for cluster in c['nOpened']:
            data[int(cluster)] = c['nOpened'][cluster]
        newsNotiScoreMap[c['_id']] = data
        
    return newsNotiScoreMap

def insertNewsNotiScoreInMongo(newsNotiSpentMap):
    for hashId in newsNotiSpentMap:
        key = {"_id" : hashId}
        data = {}
        for cluster in newsNotiSpentMap[hashId]:
            if newsNotiSpentMap[hashId][cluster][1] > 1:
                data[str(cluster)] = newsNotiSpentMap[hashId][cluster][0] / newsNotiSpentMap[hashId][cluster][1]  
        
        notiScoreData = {"$set" : {"nOpened" : data, "updatedAt" : datetime.datetime.now()}}
        newsNotiScoreCollection.update_one(key, notiScoreData, upsert=True)
        
        

In [ ]:
Log("Exectution finished successfully, in : %s minutes, %s seconds " % (int((time.time() - start_time) / 60), int((time.time() - start_time) % 60)), flag=True)